# Division

In this notebook we demonstrate the division function available within cicada. 

This is division in the conventional sense, not multiplication by a multiplicative inverse in the context of the field. Also, it is important to note that this function return as approximate quotient limited by the amount of precision given our fixed point representation. 

We use a couple fixed operands which we initialize and share. These we then use as operands for the division function. Feel free to change these operands. They are a secret dividend (secret_a) and a public divisor (b) above initialized in their own notebook cell for clarity and convenience. 

We reveal the result and calculate the error based on the result of operating on the underlying values via normal floating point arithmetic and report the results.

In [1]:
import logging

In [2]:
import numpy

In [3]:
import cicada.additive
import cicada.communicator

In [4]:
logging.basicConfig(level=logging.INFO)

In [5]:
secret_a = 37
b = 7

In [10]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    b_enc = protocol.encoder.encode(numpy.array(b, dtype=object))
    log.info(f"Player {communicator.rank} secret a: {secret_a}", src=0)
    log.info(f"Player {communicator.rank} b: {b}", src=1)
    a_share = protocol.share(src=0, secret=protocol.encoder.encode(numpy.array(secret_a)), shape=())
    quotient_share = protocol.division_private_public(a_share, b)
    quotient = protocol.encoder.decode(protocol.reveal(quotient_share))
    log.info(f"Player {communicator.rank} quotient: {quotient}")
    actual= secret_a/b
    err_percent = 100*(quotient-actual)/actual
    log.info(f"Player {communicator.rank} quotient error %: {err_percent}", src=0)


In [11]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:50533 from tcp://127.0.0.1:50533.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:50533 from tcp://127.0.0.1:50534.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:50533 from tcp://127.0.0.1:50535.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 secret a: 37
INFO:root:Player 1 b: 7
INFO:root:Player 0 quotient: 5.285552978515625
INFO:root:Player 1 quotient: 5.285552978515625
INFO:root:Player 2 quotient: 5.285552978515625
INFO:root:Player 0 quotient error %: -0.0030517578124975996
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator freed.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator freed.
INFO:

[None, None, None]